#### Importando as bibliotecas necessárias



In [1]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import requests
import json
from fuzzywuzzy import fuzz
import numpy as np

#### Lendo o arquivo com os dados

In [ ]:
dados = pd.read_excel('./Dados de estudantes da UFV - 2022-12-12.ods')
dados

#### Verificando as 5 primeiras linhas do DataFrame

In [ ]:
dados.head()
### Informação sobre o conteúdo dos dados

In [ ]:
dados.info()

#### Descrição dos dados

In [ ]:
dados.describe()

#### Verificando a quantidade de dados nulos em cada coluna

In [ ]:
dados.isnull().sum()

#### Verificando valores únicos e contagem para cada atributo

In [ ]:
for coluna in dados.columns:
    print(f"\n{coluna}\nVariedade: {dados[coluna].nunique()}\nDados: \n{dados[coluna].value_counts()}\n")

#### Copiando os dados para um novo DataFrame

In [ ]:
df = dados.copy()

### Tratamento da coluna 'Ano Nascimento'

#### Dados da coluna
    - Descrição dos dados

In [ ]:
df['Ano_Nascimento'].describe()

#### Outliers
    - Através do gráfico de caixa (boxplot) é possível verificar a presença de outliers

In [ ]:
sns.boxplot(x=df['Ano_Nascimento'], color='red')

#### Remoção de outliers
    - Para a remoção, foi considerado outliers, valores acima de 2005 e abaixo de 1930

In [ ]:
print(f"Quantidade de outliers menores que 1930: {df[df['Ano_Nascimento'] < 1930]['Ano_Nascimento'].count()}")
print(f"Quantidade de outliers maiores que 2005: {df[df['Ano_Nascimento'] > 2005]['Ano_Nascimento'].count()}")

df = df.drop(df[df['Ano_Nascimento'] < 1930].index)
df = df.drop(df[df['Ano_Nascimento'] > 2005].index)
print(f"Outliers removidos!")

sns.boxplot(x=df['Ano_Nascimento'], color='red')

#### Valores nulos
    - Através da verificação de valores nulos, foi encontrado alguns valores nulos

In [ ]:
print(f"Quantidade de valores nulos: {df['Ano_Nascimento'].isnull().sum()}")

#### Tratamento de valores nulos
    - Para o tratamento de valores nulos, foi utilizado a média dos valores da coluna

In [ ]:
df['Ano_Nascimento'].fillna(df['Ano_Nascimento'].mean(), inplace=True)
print(f"Quantidade de valores nulos: {df['Ano_Nascimento'].isnull().sum()}")

#### Tipo de dado
    - Substuituídos por inteiros

In [ ]:
df['Ano_Nascimento'] = df['Ano_Nascimento'].astype(int)

#### Distribuição dos dados
    - Ideal para uma análise inicial


In [ ]:
sns.histplot(x=df['Ano_Nascimento'], color='red', kde=True)

In [ ]:

df['Ano_Nascimento'] = df['Ano_Nascimento'].astype(int)

### Tratamento da coluna 'Sexo'

#### Dados da coluna
    - Descrição dos dados
    - Gráfico de contagem

In [ ]:
df['Sexo'].describe()

In [ ]:
df['Sexo'].value_counts()

In [ ]:
sns.countplot(x=df['Sexo'], color='red', data=df, order=df['Sexo'].value_counts().index, orient='v', palette='Set2')

#### Valores nulos
    - Valores nulos foram inicialmente substituidos por 'N/A

In [ ]:
print(f"Quantidade de valores nulos: {df['Sexo'].isnull().sum()}")

In [ ]:
df['Sexo'].fillna('N/A', inplace=True)
print(f"Quantidade de valores nulos: {df['Sexo'].isnull().sum()}")

df['Sexo'].value_counts().reset_index().rename(columns={'index': 'Sexo', 'Sexo': 'Quantidade'})

### Tratamento da coluna 'Campus'

#### Desscrição da coluna
    - Importante para verificar se possui valores sem sentido ou fora do padrão
    - Gráfico de contagem

In [ ]:
df['Campus'].describe()

In [ ]:
df['Campus'].value_counts()

In [ ]:
sns.countplot(x=df['Campus'], color='red', data=df, order=df['Campus'].value_counts().index, orient='v', palette='Set2')

#### Traamento de valores nulos
    - A coluna 'Campus' não possui valores nulos


In [ ]:
print(f"Quantidade de valores nulos: {df['Campus'].isnull().sum()}")

In [ ]:
print("A coluna 'Campus' não possui valores nulos!")

### Tratamento da coluna 'Curso_Identificador'

#### Dados da coluna
    - Descrição da coluna
    - Gráfico de contagem


In [ ]:
df['Curso_Identificador'].value_counts().reset_index().rename(columns={'index': 'Curso_Identificador', 'Curso_Identificador': 'Quantidade'})

In [ ]:
ccp = sns.countplot(x=df['Curso_Identificador'], color='red', data=df, order=df['Curso_Identificador'].value_counts().index,orient='v', palette='Set2')
ccp.set_xticklabels(ccp.get_xticklabels(), rotation=90)
ccp.figure.set_size_inches(20, 10)

#### Valores nulos
    - Tratamento de valores nulos

In [ ]:
qtdCursoIdentificadorNulos = df['Curso_Identificador'].isnull().sum()
print(f"Quantidade de valores nulos: {qtdCursoIdentificadorNulos}")

In [ ]:
print("A coluna 'Curso_Identificador' não possui valores nulos!")

#### Valores irrelevantes
    - Tratamento de valores irrelevantes

In [ ]:
# Há um curso que possui apenas 1 aluno, então iremos remover esse aluno e o curso

df[df['Curso_Identificador'] == 1423]

In [ ]:
df = df.drop(df[df['Curso_Identificador'] == 1423].index)
print("Curso removido!")

### Tratamento da coluna 'Curso'

#### Dados da coluna
    - Descrição da coluna
    - Gráfico de contagem

In [ ]:
df['Curso'].describe()

In [ ]:
ccp= sns.countplot(x=df['Curso'], color='red', data=df, order=df['Curso'].value_counts().index, orient='v', palette='Set2')
ccp.set_xticklabels(ccp.get_xticklabels(), rotation=90)
ccp.figure.set_size_inches(20, 10)

#### Nome dos cursos
    - Análise dos nomes dos cursos
    - Primeiramente foi criado um dicionário com os identificadores dos cursos e seus respectivos nomes para uma análise inicial
    - Após a análise inicial, os cursos e a quantidade de alunos por curso foram ordenados por ordem alfabética para facilitar a visualização dos dados
    - Por fim, alguns cursos foram tartados, removidos ou renomeados

In [ ]:
dict(zip(df['Curso_Identificador'], df['Curso']))

In [ ]:
df['Curso'].value_counts().sort_index().reset_index().rename(columns={'index': 'Curso', 'Curso': 'Quantidade'})

In [ ]:
# Há um curso com nome CAV - Biologia com apenas 1 aluno, então iremos remover esse aluno e o curso
df.drop(df[df['Curso'] == 'CAV - Biologia'].index, inplace=True)

# Há um curso com nome CAV - LETRAS com apenas 10 alunos, então iremos remover esse curso
df.drop(df[df['Curso'] == 'CAV - LETRAS'].index, inplace=True)

# Há um curso com nome CAV - Administração - Habilitação em Administração de Cooperativas com apenas 35 alunos, então iremos remover esse curso
df.drop(df[df['Curso'] == 'CAV - Administração - Habilitação em Administração de Cooperativas'].index, inplace=True)

#susbtituindo nome dos cursos que tem identificador: 3104 por 'CRP - Sistema de Informação - Integral'
df.loc[df['Curso_Identificador'] == 3104, 'Curso'] = 'CRP - Sistema de Informação - Integral'

#susbtituindo nome dos cursos que tem identificador: 3105 por 'CRP - Sistema de Informação - Noturno'
df.loc[df['Curso_Identificador'] == 3105, 'Curso'] = 'CRP - Sistema de Informação - Noturno'

In [ ]:
ccp = sns.countplot(x=df['Curso'], color='red', data=df, order=df['Curso'].value_counts().index, orient='v', palette='Set2')
ccp.set_xticklabels(ccp.get_xticklabels(), rotation=90)
ccp.figure.set_size_inches(20, 10)

#### Valores nulos
    - Tratamento de valores nulos

In [ ]:
print(f"Quantidade de valores nulos: {df['Curso'].isnull().sum()}")

In [ ]:
print("A coluna 'Curso' não possui valores nulos!")

### Tratamento da coluna 'UF_Nascimento'

#### Dados da coluna
    - Descrição da coluna
    - Gráfico de contagem

In [ ]:
df['UF_Nascimento'].value_counts().reset_index().rename(columns={'index': 'UF_Nascimento', 'UF_Nascimento': 'Quantidade'})

In [ ]:
ucp = sns.countplot(x=df['UF_Nascimento'], color='red', data=df, order=df['UF_Nascimento'].value_counts().index, orient='v', palette='Set2')
ucp.set_xticklabels(ucp.get_xticklabels(), rotation=90)
ucp.figure.set_size_inches(20, 10)
ucp.set_yscale('log')

#### Valores nulos
    - Tratamento de valores nulos

In [ ]:
df[df['UF_Nascimento'].isnull()]

##### Foi verificado que os alunos que não possuem UF de nascimento são alunos estrangeiros ou que não informaram o município de nascimento
##### Serão excluídos

In [ ]:
df.drop(df[df['UF_Nascimento'].isnull()].index, inplace=True)

#### Valores irrelevantes
    - Tratamento de valores irrelevantes


In [ ]:
df['UF_Nascimento'].value_counts().reset_index().rename(columns={'index': 'UF_Nascimento', 'UF_Nascimento': 'Quantidade'}).query('Quantidade < 12')

In [ ]:
df[df['UF_Nascimento'].map(df['UF_Nascimento'].value_counts()) == 3]

##### Foi verificado que alguns alunos possuem UF de nascimento vazio, estas colunas serão excluídas

In [ ]:
df = df[~df['UF_Nascimento'].isin(df['UF_Nascimento'].value_counts()[df['UF_Nascimento'].value_counts() == 3].index)]

ucp = sns.countplot(x=df['UF_Nascimento'], color='red', data=df, order=df['UF_Nascimento'].value_counts().index, orient='v', palette='Set2')
ucp.set_xticklabels(ucp.get_xticklabels(), rotation=90)
ucp.figure.set_size_inches(20, 10)
ucp.set_yscale('log')

### Tratamento da coluna 'Municipio_Nascimento'

#### Dados da coluna
    - Descrição da coluna

In [ ]:
df['Municipio_Nascimento'].value_counts().reset_index().rename(columns={'index': 'Municipio_Nascimento', 'Municipio_Nascimento': 'Quantidade'})

#### Valores nulos
    - Tratamento de valores nulos

In [ ]:
print(f"Quantidade de valores nulos: {df['Municipio_Nascimento'].isnull().sum()}")

In [ ]:
df['Municipio_Nascimento'].fillna('N/A', inplace=True)
print(f"Quantidade de valores nulos: {df['Municipio_Nascimento'].isnull().sum()}")

#### Tipo de dado
    - Transformando o tipo de dado para string

In [ ]:
df['Municipio_Nascimento'] = df['Municipio_Nascimento'].astype(str)

#### Nome dos municipios
    - Tratamento de nomes de municipios

***Removendo os espaços em branco no início e no fim de cada nome de municipio***

In [ ]:
df['Municipio_Nascimento'] = df['Municipio_Nascimento'].str.strip()

***Removendo os espaços em branco duplicados no meio de cada nome de municipio***

In [ ]:
df['Municipio_Nascimento'] = df['Municipio_Nascimento'].str.replace(' +', ' ', regex=True)

***Municipios que estão em uppercase serão colocados em lower-case, com exceção da suas iniciais***

In [ ]:
if df['Municipio_Nascimento'].str.isupper().any():
    df['Municipio_Nascimento'] = df['Municipio_Nascimento'].str.lower()
    df['Municipio_Nascimento'] = df['Municipio_Nascimento'].str.title()

***Colocando todas as cidades em uma lista***

In [ ]:
munUFV = df['Municipio_Nascimento'].unique().tolist()
len(munUFV)

***Lista de municípios do IBGE - Tratamento***

In [ ]:
url = 'https://servicodados.ibge.gov.br/api/v1/localidades/municipios'
r = requests.get(url)
munExistente = json.loads(r.text)
munExistente = [municipio['nome'] for municipio in munExistente]
munExistente

***Verificando se os municípios existem na realidade***

In [ ]:
munNaoExistente = []
for municipio in munUFV:
    if municipio not in munExistente:
        munNaoExistente.append(municipio)
print(f'Quantidade de municípios que não estão na lista do IBGE: {len(munNaoExistente)}')

***Tratamento dos possíveis municipios que não existem. Similaridades acima de 90 entre os municipios que não existem e os municipios do IBGE***
***usando a biblioteca fuzzywuzzy, serão substituídos***

In [ ]:
munSim = []

for municipio in munNaoExistente:
    for municipioIBGE in munExistente:
        if fuzz.ratio(municipio, municipioIBGE) > 90:
            munSim.append((municipio, municipioIBGE))
print(f'Quantidade de municípios que não estão na lista do IBGE: {len(munSim)}')

***Usando similaridades para substituir os municipios que não existem para os municipios do IBGE***

In [ ]:
for municipio in munSim:
    df['Municipio_Nascimento'] = df['Municipio_Nascimento'].str.replace(municipio[0], municipio[1])

In [ ]:
for municipio in munSim:
    if municipio[0] in munNaoExistente:
        munNaoExistente.remove(municipio[0])
print(f'Quantidade de municípios que não estão na lista do IBGE: {len(munNaoExistente)}')

***O restante dos municipios que não existem serão excluídos, pois não possuem similaridades com os municipios do IBGE ou são de outros países***

In [ ]:
for municipio in munNaoExistente:
    df.drop(df[df['Municipio_Nascimento'] == municipio].index, inplace=True)

In [ ]:
df['Municipio_Nascimento'].value_counts()

In [ ]:
mun10 = df['Municipio_Nascimento'].value_counts().nlargest(10)

sns.barplot(x=mun10.index, y=mun10.values, palette='Set2')
plt.xticks(rotation=90)
plt.title('Top 10 Municípios de Nascimento')
plt.show()

### Tratamento da coluna 'Admissao'

#### Dados da coluna
    - Tipo e quantidade de dados

In [ ]:
print("Tipo de dado: ", df['Admissao'].dtype)
print("Quantidade de dados: ", df['Admissao'].shape[0])

#### Valores nulos
    - Tratamento de valores nulos

In [ ]:
print(f"Quantidade de valores nulos: {df['Admissao'].isnull().sum()}")

#### Tratamento
    - Valores inválidos

##### Algumas admissões são inválidas. As que posssuem valores 0 no campo MM serão substituídas por 01 (YYYY/00 --> YYYY/01)

In [ ]:
df['Admissao'] = df['Admissao'].str.replace('/0', '/1')
df['Admissao'].value_counts()

#### Visualização
    - Histograma
    - Mais admissões

In [ ]:
ahp = sns.histplot(x=df['Admissao'], color='red', data=df, bins=20, kde=True)
ahp.set_xticklabels(ahp.get_xticklabels(), rotation=90)
ahp.figure.set_size_inches(15, 10)
plt.title('Admissões')
plt.show()

In [ ]:
df['Admissao'].value_counts().reset_index().rename(columns={'index': 'Admissao', 'Admissao': 'Quantidade'})

### Tratamento da coluna 'Saida'

#### Dados da coluna
    - Tipo e quantidade de dados

In [ ]:
print("Tipo de dado: ", df['Saida'].dtype)
print("Quantidade de dados: ", df['Saida'].shape[0])
df['Saida'].value_counts()

#### Valores nulos
    - Tratamento de valores nulos

In [ ]:
print(f"Quantidade de valores nulos: {df['Saida'].isnull().sum()}")

#### Tratamento
    - Valores inválidos

##### Algumas saídas são inválidas. As que posssuem valores 0 no campo MM serão substituídas por 01 (YYYY/00 --> YYYY/01)
##### A que possui 0, será substituída por 'CURSANDO', pois o aluno ainda está cursando

In [ ]:
df['Saida'] = df['Saida'].replace(0, 'CURSANDO')
df['Saida'] = df['Saida'].str.replace('/0', '/1')

In [ ]:
df['Saida'].value_counts().reset_index().rename(columns={'index': 'Saida', 'Saida': 'Quantidade'})

### Visualização
    - Histograma
    - Mais saídas

In [ ]:
shp = sns.histplot(x=df['Saida'], color='red', data=df, bins=20, kde=True)
shp.set_xticklabels(shp.get_xticklabels(), rotation=90)
shp.figure.set_size_inches(15, 10)
plt.title('Saídas')
plt.show()

In [ ]:
df['Saida'].value_counts().reset_index().rename(columns={'index': 'Saida', 'Saida': 'Quantidade'})

### Tratamento da coluna 'Codigo_Situacao_Aluno'

#### Dados da coluna
    - Valores únicos
    - Agrpados com Situação do Aluno

In [ ]:
print("Valores únicos: ", df['Codigo_Situacao_Aluno'].unique())
print("Quantidade de valores únicos: ", df['Codigo_Situacao_Aluno'].nunique())

In [ ]:
df.groupby(['Codigo_Situacao_Aluno', 'Situacao_Aluno']).size().reset_index().rename(columns={0: 'Quantidade'})

#### Valores nulos
    - Tratamento de valores nulos

In [ ]:
print(f"Quantidade de valores nulos: {df['Codigo_Situacao_Aluno'].isnull().sum()}")

#### Tratamento
    - Valores repetidos
    - Remoção de colunas

In [ ]:
# Código: A --> Situação: Abandono

df.loc[df['Codigo_Situacao_Aluno'] == 'A', 'Situacao_Aluno'] = 'Abandono'

# Código: B --> Situação: Conclusão (Não Colou Grau)

df.loc[df['Codigo_Situacao_Aluno'] == 'B', 'Situacao_Aluno'] = 'Conclusão/Não Colou Grau'

# Código: Z --> Situação: Trancamento por Motivo de Saúde

df.loc[df['Codigo_Situacao_Aluno'] == 'Z', 'Situacao_Aluno'] = 'Trancamento por Motivo de Saúde'

df.groupby(['Codigo_Situacao_Aluno', 'Situacao_Aluno']).size().reset_index().rename(columns={0: 'Quantidade'})

##### Situações que possuem menos de 8 alunos serão excluídas, pois não possuem relevância para a análise

In [ ]:
situacaoAluno = df['Situacao_Aluno'].value_counts().reset_index().rename(columns={'index': 'Situacao_Aluno', 'Situacao_Aluno': 'Quantidade'})
situacaoAluno = situacaoAluno[situacaoAluno['Quantidade'] >= 8]
situacaoAluno

In [ ]:
df.drop(df[~df['Situacao_Aluno'].isin(situacaoAluno['Situacao_Aluno'])].index, inplace=True)

In [ ]:
df.groupby(['Codigo_Situacao_Aluno', 'Situacao_Aluno']).size().reset_index().rename(columns={0: 'Quantidade'})

#### Visualização
    - Gráfico de Contagem

In [ ]:
sns.set_palette("Blues")
sns.countplot(x='Codigo_Situacao_Aluno', data=df, order=df['Codigo_Situacao_Aluno'].value_counts().index)
plt.title('Situação do Aluno')
plt.yscale('log')
plt.show()

### Tratamento da coluna 'Situacao_Aluno'

#### Dados da coluna
    - Valores únicos


In [ ]:
print("Valores únicos: ", df['Situacao_Aluno'].unique())

#### Valores nulos
    - Tratamento de valores nulos

In [ ]:
print(f"Quantidade de valores nulos: {df['Situacao_Aluno'].isnull().sum()}")

#### Tratamento
    - Remoção de espaços vazios

In [ ]:
df['Situacao_Aluno'] = df['Situacao_Aluno'].str.strip()
df['Situacao_Aluno'].value_counts().reset_index().rename(columns={'index': 'Situacao_Aluno', 'Situacao_Aluno': 'Quantidade'})

#### Visualização
    - Gráfico de Contagem

In [ ]:
scp = sns.countplot(x='Situacao_Aluno', data=df, order=df['Situacao_Aluno'].value_counts().index)
scp.set_xticklabels(scp.get_xticklabels(), rotation=90)
scp.figure.set_size_inches(10, 6)
plt.title('Situação do Aluno')
plt.yscale('log')
plt.show()



### Tratamento da coluna 'Situacao_Aluno_Agrupada'

#### Dados da coluna
    - Valores únicos


In [ ]:
print("Valores únicos: ", df['Situacao_Aluno_Agrupada'].unique())

#### Valores nulos
    - Tratamento de valores nulos

In [ ]:
print(f"Quantidade de valores nulos: {df['Situacao_Aluno_Agrupada'].isnull().sum()}")

In [ ]:
df['Situacao_Aluno_Agrupada'].value_counts().reset_index().rename(columns={'index': 'Situacao_Aluno_Agrupada', 'Situacao_Aluno_Agrupada': 'Quantidade'})

#### Visualização
    - Gráfico de Contagem

In [ ]:
sns.set_palette("RdBu")
sns.countplot(x='Situacao_Aluno_Agrupada', data=df, order=df['Situacao_Aluno_Agrupada'].value_counts().index)
plt.title('Situação do Aluno Agrupada')
plt.yscale('log')
plt.show()

### Tratamento da coluna 'Modalidade_Inscrita'

#### Dados da coluna
    - Valores únicos
    - Quantidade de alunos
    - Tipo de dado

In [ ]:
print("Valores únicos: ", df['Modalidade_Inscrita'].unique())
print(f"Tipo de dado: {df['Modalidade_Inscrita'].dtype}")

In [ ]:
df['Modalidade_Inscrita'].value_counts().reset_index().rename(columns={'index': 'Modalidade_Inscrita', 'Modalidade_Inscrita': 'Quantidade'})

##### Valores nulos não serão tratados, pois não possuem relevância para a análise

In [ ]:
print(f"Quantidade de valores nulos: {df['Modalidade_Inscrita'].isnull().sum()}")

In [ ]:
df['Modalidade_Inscrita'].value_counts().reset_index().rename(columns={'index': 'Modalidade_Inscrita', 'Modalidade_Inscrita': 'Quantidade'})

#### Visualização
    - Gráfico de Contagem

In [ ]:
sns.countplot(x='Modalidade_Inscrita', data=df, order=df['Modalidade_Inscrita'].value_counts().index)
plt.title('Modalidade Inscrita')
plt.yscale('log')
plt.show()

#### Tratamento da coluna 'ENEM

#### Dados da coluna
    - Descrição dos dados
    - Quantidade de alunos
    - Tipo de dado

In [ ]:
print(f"Tipo de dado: {df['ENEM'].dtype}")
df['ENEM'].describe()

In [ ]:
df['ENEM'].value_counts().reset_index().rename(columns={'index': 'ENEM', 'ENEM': 'Quantidade'})

#### Valores nulos
    - Tratamento de valores nulos

##### Valores nulos não serão tratados, pois não possuem relevância para a análise

In [ ]:
print(f"Quantidade de valores nulos: {df['ENEM'].isnull().sum()}")

#### Visualização
    - Boxplot
    - Violinplot
    - Histograma
    - Boxplot por Modalidade_Inscrita
    - BoxPlot por CRA
    - Violinplot por Modalidade_Inscrita
    - Correlação entre ENEM e Modalidade_Inscrita

In [ ]:
sns.boxplot(x='ENEM', data=df)
plt.title('ENEM')
plt.show()

In [ ]:
sns.violinplot(x='ENEM', data=df)
plt.title('ENEM')
plt.show()


In [ ]:
plt.hist(df['ENEM'], bins=50, edgecolor='black' )
plt.xlabel('Nota do ENEM')
plt.ylabel('Frequência')
plt.title('Distribuição das Notas do ENEM')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x='Modalidade_Inscrita', y='ENEM', data=df)
plt.title('ENEM')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.violinplot(x='Modalidade_Inscrita', y='ENEM', data=df)
plt.title('ENEM')
plt.show()

In [ ]:
sns.heatmap(df[['ENEM', 'Modalidade_Inscrita']].corr(), annot=True)

#### Tratamento da coluna 'CRA'

#### Dados da coluna
    - Descrição dos dados
    - Quantidade de alunos
    - Tipo de dado

In [ ]:
print(f"Tipo de dado: {df['CRA'].dtype}")
df['CRA'].describe()

In [ ]:
df['CRA'].value_counts().reset_index().rename(columns={'index': 'CRA', 'CRA': 'Quantidade'})

#### Valores nulos
    - Tratamento de valores nulos

##### Valores nulos não serão tratados, pois não possuem relevância para a análise

In [ ]:
print(f"Quantidade de valores nulos: {df['CRA'].isnull().sum()}")

####  Outliers
    - Tratamento de outliers

##### Valor 0 será substituído por NaN

In [ ]:
df['CRA'] = df['CRA'].replace(0, np.nan)

In [ ]:
df['CRA'].describe()

In [ ]:
df['CRA'].value_counts().reset_index().rename(columns={'index': 'CRA', 'CRA': 'Quantidade'})

#### Visualização
    - Boxplot
    - Violinplot
    - Histograma
    - Boxplot por Modalidade_Inscrita
    - Matriz de dispersão

In [ ]:
sns.boxplot(x='CRA', data=df)
plt.title('CRA')
plt.show()

In [ ]:
sns.violinplot(x='CRA', data=df)
plt.title('CRA')
plt.show()

In [ ]:
plt.hist(df['CRA'], bins=50, edgecolor='black' )
plt.xlabel('CRA')
plt.ylabel('Frequência')
plt.title('Distribuição do CRA')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x='Modalidade_Inscrita', y='CRA', data=df)
plt.title('CRA')
plt.show()

In [ ]:
cmd = df[['ENEM', 'CRA', 'Modalidade_Inscrita']]
plt.figure(figsize=(10, 8))
g = sns.pairplot(cmd, hue='Modalidade_Inscrita')
g.fig.set_size_inches(10, 8)
plt.show()

### Tratamento da coluna 'Num_Reprovacoes'

#### Dados da coluna
    - Descrição dos dados
    - Quantidade de alunos
    - Tipo de dado

In [ ]:
print(f"Tipo de dado: {df['Num_Reprovacoes'].dtype}")
df['Num_Reprovacoes'].describe()

In [ ]:
df['Num_Reprovacoes'].value_counts().reset_index().rename(columns={'index': 'Num_Reprovacoes', 'Num_Reprovacoes': 'Quantidade'})

#### Valores nulos
    - Tratamento de valores nulos

In [ ]:
print(f"Quantidade de valores nulos: {df['Num_Reprovacoes'].isnull().sum()}")

#### Visualização
    - Countplot
    - Boxplot
    - BoxPlot por Modalidade_Inscrita
    - Heatmap
    - Matriz de Dispersão

In [ ]:
plt.figure(figsize=(15, 6))
sns.countplot(x='Num_Reprovacoes', data=df, order=df['Num_Reprovacoes'].value_counts().index)
plt.title('Número de Reprovações')
plt.yscale('log')

plt.show()

In [ ]:
sns.boxplot(x='Num_Reprovacoes', data=df)
plt.title('Número de Reprovações')
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x='Modalidade_Inscrita', y='Num_Reprovacoes', data=df)
plt.title('Número de Reprovações')
plt.show()

##### Não a correlação entre reprovações e modalidade do ENEM

In [ ]:
sns.heatmap(df[['Num_Reprovacoes', 'Modalidade_Inscrita']].corr(), annot=True)

In [ ]:
rmd = df[['ENEM', 'CRA', 'Modalidade_Inscrita', 'Num_Reprovacoes']]
plt.figure(figsize=(10, 8))
g = sns.pairplot(rmd, hue='Modalidade_Inscrita')
g.fig.set_size_inches(10, 8)
plt.show()

##### Achei estranho não ter correlação entre o CRA e o número de reprovações, pelo menos não com uma análise mais simples

In [ ]:
df[['Num_Reprovacoes', 'CRA']].corr()

### Tratamento da coluna 'Raca'

#### Dados da coluna
    - Descrição dos dados
    - Unicos
    - Tipo de dado

In [ ]:
print(f"Tipo de dado: {df['Raca'].dtype}")
print("Unicos: ", df['Raca'].unique())

In [ ]:
df['Raca'].value_counts().reset_index().rename(columns={'index': 'Raca', 'Raca': 'Quantidade'})

#### Valores nulos
    - Tratamento de valores nulos

In [ ]:
print(f"Quantidade de valores nulos: {df['Raca'].isnull().sum()}")

In [ ]:
df['Raca'].fillna('Não Informado', inplace=True)

##### A princípio, valores Informação não disponível serão tratados como NaN

In [ ]:
df['Raca'] = df['Raca'].replace('Informação não disponível', np.nan)

In [ ]:
print(f"Quantidade de valores nulos: {df['Raca'].isnull().sum()}")

In [ ]:
df['Raca'].value_counts().reset_index().rename(columns={'index': 'Raca', 'Raca': 'Quantidade'})

#### Visualização
    - Countplot
    - Boxplots
    - Correlação


In [ ]:
plt.figure(figsize=(15, 6))
sns.countplot(x='Raca', data=df, order=df['Raca'].value_counts().index)
plt.title('Raça')
plt.yscale('log')

##### Fica claro a relação de Modalidade_Inscrita com Raça

In [ ]:
plt.figure(figsize=(15, 6))
sns.countplot(x='Raca', data=df, order=df['Raca'].value_counts().index, hue='Modalidade_Inscrita')
plt.title('Raça')
plt.yscale('log')

##### Bem dividido M e F

In [ ]:
plt.figure(figsize=(15, 6))
sns.countplot(x='Raca', data=df, order=df['Raca'].value_counts().index, hue='Sexo')
plt.title('Raça')

In [ ]:
plt.figure(figsize=(15, 6))
sns.boxplot(x='Raca', y='CRA', data=df, order=df['Raca'].value_counts().index)
plt.title('Raça')

In [ ]:
plt.figure(figsize=(15, 6))
sns.boxplot(x='Raca', y='CRA', data=df, order=df['Raca'].value_counts().index, hue='Modalidade_Inscrita')
plt.title('Raça')


##### Mulheres parece ter CRA maior que homens

In [ ]:
plt.figure(figsize=(15, 6))
sns.boxplot(x='Raca', y='CRA', data=df, order=df['Raca'].value_counts().index, hue='Sexo')
plt.title('Raça')


In [ ]:
plt.figure(figsize=(15, 6))
sns.boxplot(x='Raca', y='ENEM', data=df, order=df['Raca'].value_counts().index, hue='Sexo')
plt.title('Raça')